In [1]:
import pandas as pd

In [2]:
df = pd.read_json('database-bil0-1.json')

In [3]:
df.drop(['آژانس املاک', 'آگهی‌دهنده', 'نوع ساختمان', 'تور مجازی ۳۶۰ درجه'], axis=1, inplace=True)

In [4]:
#create goal DataFrame: df_detail

df_detail = df[df['جزیئات']==True]
df_detail.reset_index(inplace=True, drop=True)

In [5]:
#utility

utility_true_false = ['بالکن','ایرانی','فرنگی']
utility_list = ['تأمین\u200cکننده آب گرم','جنس کف','سرمایش','گرمایش']

In [6]:
def utility_extract(df, utility_list, utility_true_false):
    for i in range (df.shape[0]):
        if df.iloc[i, 12]:
            for line in df.iloc[i, 15]:
                for obj in utility_list:
                    if obj in line:
                        df.loc[i, obj] = line.replace(obj, '').strip()       
                for obj in utility_true_false:
                    df.loc[i, obj] = False if 'ندارد' in line else True

utility_extract(df_detail, utility_list, utility_true_false)

C:\Users\mehrh\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\mehrh\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [7]:
def year2num(txt:str):
    if txt.isnumeric():
        return int(txt)
    else:
        return 1370

df_detail['ساخت'] = df_detail.agg({'ساخت':year2num})

C:\Users\mehrh\AppData\Local\Temp/ipykernel_8676/1773767381.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_detail['ساخت'] = df_detail.agg({'ساخت':year2num})


In [8]:
def room2num(txt:str):
    if txt.isnumeric():
        return int(txt)
    else:
        if len(txt.split())==2:
            return 0
        else:
            return int(txt[1])

df_detail['اتاق'] = df_detail.agg({'اتاق':room2num})

C:\Users\mehrh\AppData\Local\Temp/ipykernel_8676/2769779421.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_detail['اتاق'] = df_detail.agg({'اتاق':room2num})


In [9]:
def choose_floor(txt):
    split = txt.split(' از ')[0]
    if split.isnumeric():
        split = int(split)
    else:
        split = {'زیرهمکف':-1,
                 'همکف':0,
                 '+۳۰':31,
                 '۳۰+':31}[split]
    return split

df_detail['طبقه واحد'] = df_detail['طبقه'].apply(lambda x: choose_floor(x))

C:\Users\mehrh\AppData\Local\Temp/ipykernel_8676/1875224094.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_detail['طبقه واحد'] = df_detail['طبقه'].apply(lambda x: choose_floor(x))


In [10]:
#Change dtypes

df_detail['متراژ'] = df_detail['متراژ'].astype('int')
df_detail['ناحیه'] = df_detail['ناحیه'].astype('category')

be_category=['سند','وضعیت واحد','جهت ساختمان','ناحیه','تعداد واحد در طبقه','گرمایش','سرمایش','جنس کف','تأمین‌کننده آب گرم',]
for col in be_category:
    if df_detail[col].dtype == 'O':
        df_detail[col] = df_detail[col].astype('category')
        
True_false_columns = ['بالکن','ایرانی','فرنگی','آسانسور','پارکینگ','انباری',]
for obj in True_false_columns:
    df_detail[obj] = df_detail[obj].astype('bool')

for col in ['قیمت کل', 'قیمت هر متر']:
    df_detail[col] =  df_detail[col].str.replace('٫', '').str.replace('تومان', '').dropna().astype('int64')
for col in ['قیمت کل', 'قیمت هر متر']:
    df[col] =  df[col].str.replace('٫', '').str.replace('تومان', '').dropna().astype('int64')

C:\Users\mehrh\AppData\Local\Temp/ipykernel_8676/1249932701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_detail['متراژ'] = df_detail['متراژ'].astype('int')
C:\Users\mehrh\AppData\Local\Temp/ipykernel_8676/1249932701.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_detail['ناحیه'] = df_detail['ناحیه'].astype('category')
C:\Users\mehrh\AppData\Local\Temp/ipykernel_8676/1249932701.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [11]:
#Drop and Create the final DataFrame

df_detail.drop(['امکانات', 'جزیئات'], inplace=True, axis=1)
df_detail.drop_duplicates(subset='token', inplace=True)
# df_detail.dropna(inplace=True)

C:\Users\mehrh\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\mehrh\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [12]:
#Matrices

MAT_X = df_detail.drop(['طبقه', 'token', 'متن توضیحات','قیمت کل', 'قیمت هر متر'], axis=1)
VEC_Y = df_detail['قیمت کل']
VEC_Y2 = df_detail['قیمت هر متر']

In [13]:
#To pickle

MAT_X.to_pickle('mat_x.pkl')
VEC_Y.to_pickle('vec_y.pkl')
VEC_Y2.to_pickle('vec_y2.pkl')

In [14]:
# test = pd.read_pickle('mat_x.pkl')